# Poject: Sequential Pattern Mining with Web log

In this project we will use spark and the prefix algorithm to mine a web log from an online store.


Author: Kenny Zhang

E-mail: tzhang808@gmail.com

Date: Apr 3, 2023

## About Data and Cleaning

**Dataset**: https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/3QBYB5

- The above link serves as explaination of the dataset. You can download data from https://cis331.guihang.org/data/access.log.zip
- **Download and unzip** the file `Access.log.zip` ; Move the file `access.log` to the directory that you map into docker instance:

Say, I move my file into my home: `/Users/zhengqu` ; then in Mac terminal/Windows command prompt, I "cd" to my home dir (just type "cd" and enter, it will do).

```bash
pwd
```

<font size=-1 color=grey>/Users/zhengqu</font>

Let's assume I put `access.log` in my home dir as `/Users/zhengqu/access.log` and my spark docker container/instance was created by the following command, as I demonstrated in the video after the spark lab lecture:

```bash
docker run -p 10000:8888 -d -P --name notebook -it  -v "$PWD:/home/jovyan/mydir" jupyter/all-spark-notebook
```

whereby `"$PWD"` is actually `/Users/zhengqu` in my case, and it is mapped into docker container `notebook` as `/home/jovyan/mydir`. So anything I put in this folder `"/Users/zhengqu"` on my host laptop is available in my spark docker container `notebook` in this folder: `/home/jovyan/mydir`. 



### Access the data file from docker container

- First you start the spark docker contaner/instance (you don't have to if your docker container is not stopped since last running) 

Use this command to show docker container running status:

```bash
docker ps -a
```

- To start /start container:

```bash
docker start notebook
```

- Next you connect your container with bash:

```bash
# connect to docker container with bash
docker exec -it notebook /bin/bash
```

- Now you are inside the spark docker container (imagine it as a virtual linux machine).

----
**Note: commands below are run from docker container linux system**


- Let's "cd" to the mapped folder:

```bash
# change to mapped dir:
cd /home/jovyan/mydir
```


- Let's check the file by running: 

```bash
 ls -l access.log
```
<font size=-2 color=grey>`-rw-r--r--@ 1 jovyan  staff 3502440823 Jan 26  2019 access.log`</font>

- Let's check how many lines it has by running (this takes a while):

```bash
wc -l access.log
```
<font size=-2 color=grey>`10365152 access.log`</font>

- Let's print the last 100 lines from end of the file:

```bash
tail -n 100 access.log
```



<font size=-2 color=#524d43>

```
151.239.2.90 - - [26/Jan/2019:20:29:09 +0330] "GET /static/bundle-bundle_site_head.css HTTP/1.1" 499 0 "https://www.zanbil.ir/search/%D8%B3%D8%B1%D8%B1%D8%B3%DB%8C%D8%AF/p0" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:64.0) Gecko/20100101 Firefox/64.0" "-"
151.239.2.90 - - [26/Jan/2019:20:29:09 +0330] "GET /image/%7B%7BbasketItem.id%7D%7D?type=productModel&wh=50x50 HTTP/1.1" 200 5 "https://www.zanbil.ir/search/%D8%B3%D8%B1%D8%B1%D8%B3%DB%8C%D8%AF/p0" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:64.0) Gecko/20100101 Firefox/64.0" "-"
5.114.175.169 - - [26/Jan/2019:20:29:09 +0330] "GET /image/56602/productModel/200x200 HTTP/1.1" 200 6188 "https://www.zanbil.ir/m/filter/b2%2Cp65%2Ct31" "Mozilla/5.0 (iPhone; CPU iPhone OS 11_4_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/11.0 Mobile/15E148 Safari/604.1" "-"
5.114.175.169 - - [26/Jan/2019:20:29:09 +0330] "GET /image/62175/productModel/200x200 HTTP/1.1" 200 8140 "https://www.zanbil.ir/m/filter/b2%2Cp65%2Ct31" "Mozilla/5.0 (iPhone; CPU iPhone OS 11_4_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/11.0 Mobile/15E148 Safari/604.1" "-"
5.114.175.169 - - [26/Jan/2019:20:29:09 +0330] "GET /image/62064/productModel/200x200 HTTP/1.1" 200 6840 "https://www.zanbil.ir/m/filter/b2%2Cp65%2Ct31" "Mozilla/5.0 (iPhone; CPU iPhone OS 11_4_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/11.0 Mobile/15E148 Safari/604.1" "-"
151.239.2.90 - - [26/Jan/2019:20:29:09 +0330] "GET /static/images/search-category-arrow.png HTTP/1.1" 200 217 "https://znbl.ir/static/bundle-bundle_site_head.css" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:64.0) Gecko/20100101 Firefox/64.0" "-"
[truncated]
```
</font>

---

## Understand the Web Log Format

You want to read [this article](https://www.sumologic.com/blog/apache-access-log/) to understand the web log format. Each web site may have a slightly different format. In general, you will see a user IP at the beginning, then the timestamp (in `[ ]` in above case) and user request (in simplified terms, sending data is called `POST`, like send login password; retrieving data is called `GET`). The request URL doesn't contain the base domain name. So 

```
GET /image/62175/productModel/200x200
```

means retrieving https://www.zanbil.ir/mage/62175/productModel/200x200
(the base domain name of this web log is www.zanbil.ir)

The 3 digits number after `HTTP/1.1` is the access status: 200 means success,` 3** means success but user previously requested the same data so it might be served from browser cache (saved data in user browser), 4** means client (user browser) error (404 is the infamous non-existing status); 5** means server error. <font color="orange">So you only care about 200 and 3** status, and want to filter out lines with other status types.</font>

You will see each IP at one timestamp requesting data from many URLs simultaneously--that's because when you click a page, multiple images, css, javascript files within that page will be retrieved and each leaves a separate web log entry. 


<font color="orange">You want to remember before mining further you will filter out all lines containing requesting css or js (you want to manually figure out patterns for these types of urls) which are normally useless for our purpose.</font>

You will also notice some log entry contain this type of lines:

<font size=-2 color=#888>
```
37.129.59.160 - - [26/Jan/2019:20:29:12 +0330] "GET /basket/add/62424?mobile=1&addedValues= HTTP/1.1" 302 0 "https://www-zanbil-ir.cdn.ampproject.org/v/s/www.zanbil.ir/m/product/32148/%DA%AF%D9%88%D8%B4%DB%8C-%D8%AA%D9%84%D9%81%D9%86-%D8%A8%DB%8C-%D8%B3%DB%8C%D9%85-%D9%BE%D8%A7%D9%86%D8%A7%D8%B3%D9%88%D9%86%DB%8C%DA%A9-%D9%85%D8%AF%D9%84-Panasonic-Cordless-Telephone-KX-TGC412?amp_js_v=0.1&usqp=mq331AQECAEoAQ%3D%3D" "Mozilla/5.0 (Linux; Android 6.0.1; D6633 Build/23.5.A.1.291) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Mobile Safari/537.36" "-"
```
</font>

- Note: <font color="orange">this URL `/basket/add/62424` means the user's action to add a product to the basket!</font>


---

### Preprocess Tasks 

To simplify our task, we will only consider users who have added at least a product to the basket. So preprocess steps involve:

0. read the access.log into spark`*`
1. filter out all unnecessary lines as highlighted in orange before.
2. Get a list of IPs (let's call it uniqueIPs) who performed the action of adding product(s) to the basket 
3. filter out all records whose IP does not belong to the IP list in step 2.

The remaining records are what we will mine with prefixspan.

`*` For step 0, for reading text file into spark, follow examples on this page (click python version): https://spark.apache.org/docs/latest/mllib-frequent-pattern-mining.html


### Example on a notebook running from docker container

- Run this from docker container's Linux bash command line to show jupyter notebook tokens

```
jupyter notebook list
```

<font size=-2 color=grey>

```
Currently running servers:
http://0.0.0.0:8888/?token=01e38a2234531e653d320df569dbc94173ebe89460cd57a :: /home/jovyan
```
</font>


In your laptop browser you open this url:
`http://127.0.0.1:10000/?token=01e38a2234531e653d320df569dbc94173ebe89460cd57a`

Note you replace 8888 with 10000 , then you navigate to your mapped dir which is `mydir` if you follow above commands. Then you do "file" and new notebook, save the new notebook (file>save notebook). You will also see the new notebook in your laptop folder, in my case it's in `/Users/zhengqu` -- this notebook is running in docker container, so do not simultaneously open the same notebook in host laptop (Windows/MAC)'s jupyter! This creates conflict.

#### Let's see pyspark example from https://spark.apache.org/docs/latest/mllib-frequent-pattern-mining.html


```python
from pyspark.mllib.fpm import FPGrowth
from pyspark import SparkContext 
sc = SparkContext('local[*]')

# Note data is an object of RDD: pyspark.rdd.PipelinedRDD
data = sc.textFile("data/mllib/sample_fpgrowth.txt") 
transactions = data.map(lambda line: line.strip().split(' '))
model = FPGrowth.train(transactions, minSupport=0.2, numPartitions=10)
result = model.freqItemsets().collect() ## << note .collect() 
for fi in result:
    print(fi)
```

Please run this example. Note 
- you need to download `sample_fpgrowth.txt` from 
https://github.com/apache/spark/raw/master/data/mllib/sample_fpgrowth.txt
and properly specify the file path in `sc.textFile()`. You can use `!wget URL-to-data` in jupyter notebook, then `!ls path/to/downloaded/file` to verify the file
- The spark object collection (data frame or pipelines) cannot be iterated until be applied with `.collect()` as in above example or using  `.take(n)` as in demo in class.

## <font color=red>You code to run example of sample_fpgrowth.txt with FPGrowth here </font>



In [6]:
!ls sample_fpgrowth.txt

sample_fpgrowth.txt


In [7]:
from pyspark.mllib.fpm import FPGrowth
from pyspark import SparkContext 
# sc = SparkContext('local[*]')
sc = SparkContext.getOrCreate();

# Note data is an object of RDD: pyspark.rdd.PipelinedRDD
data = sc.textFile("sample_fpgrowth.txt") 
transactions = data.map(lambda line: line.strip().split(' '))
model = FPGrowth.train(transactions, minSupport=0.2, numPartitions=10)
result = model.freqItemsets().collect() ## << note .collect() 
for fi in result:
    print(fi)

FreqItemset(items=['z'], freq=5)
FreqItemset(items=['x'], freq=4)
FreqItemset(items=['x', 'z'], freq=3)
FreqItemset(items=['y'], freq=3)
FreqItemset(items=['y', 'x'], freq=3)
FreqItemset(items=['y', 'x', 'z'], freq=3)
FreqItemset(items=['y', 'z'], freq=3)
FreqItemset(items=['r'], freq=3)
FreqItemset(items=['r', 'x'], freq=2)
FreqItemset(items=['r', 'z'], freq=2)
FreqItemset(items=['s'], freq=3)
FreqItemset(items=['s', 'y'], freq=2)
FreqItemset(items=['s', 'y', 'x'], freq=2)
FreqItemset(items=['s', 'y', 'x', 'z'], freq=2)
FreqItemset(items=['s', 'y', 'z'], freq=2)
FreqItemset(items=['s', 'x'], freq=3)
FreqItemset(items=['s', 'x', 'z'], freq=2)
FreqItemset(items=['s', 'z'], freq=2)
FreqItemset(items=['t'], freq=3)
FreqItemset(items=['t', 'y'], freq=3)
FreqItemset(items=['t', 'y', 'x'], freq=3)
FreqItemset(items=['t', 'y', 'x', 'z'], freq=3)
FreqItemset(items=['t', 'y', 'z'], freq=3)
FreqItemset(items=['t', 's'], freq=2)
FreqItemset(items=['t', 's', 'y'], freq=2)
FreqItemset(items=['t', '

Note:

- data is an object of RDD: 
```python
type(data)
```
> <font size=-2>pyspark.rdd.PipelinedRDD</font>
- `sc.textFile()` takes the path to your input file. Use jupyter command `!ls path/to/access.log` to verify your file path. Or switch to docker container command line, using something like `ls /home/jovyan/mydir/access.log` to verify.
- `data.map()` takes a function to be applied to each line in the input text file. If you prefer not using lambda function, you can define a named function like:
```python
def myParser (line): 
   return line.strip().split(' ')
```

And then you can use `data.map` like:
```python
transactions = data.map(myParser)
```

Please try it in the example. Also reference notebook "pyspark-ex.ipynb" used in my video demo for docker/spark.


## <font color=red>You code to run example of sample_fpgrowth.txt with myParser here </font>

In [8]:
def myParser1 (line): 
   return line.strip().split(' ')

sc = SparkContext.getOrCreate();

data = sc.textFile("sample_fpgrowth.txt") 
transactions = data.map(myParser1)
model = FPGrowth.train(transactions, minSupport=0.2, numPartitions=10)
result = model.freqItemsets().collect() ## << note .collect() 
for fi in result:
    print(fi)

FreqItemset(items=['z'], freq=5)
FreqItemset(items=['x'], freq=4)
FreqItemset(items=['x', 'z'], freq=3)
FreqItemset(items=['y'], freq=3)
FreqItemset(items=['y', 'x'], freq=3)
FreqItemset(items=['y', 'x', 'z'], freq=3)
FreqItemset(items=['y', 'z'], freq=3)
FreqItemset(items=['r'], freq=3)
FreqItemset(items=['r', 'x'], freq=2)
FreqItemset(items=['r', 'z'], freq=2)
FreqItemset(items=['s'], freq=3)
FreqItemset(items=['s', 'y'], freq=2)
FreqItemset(items=['s', 'y', 'x'], freq=2)
FreqItemset(items=['s', 'y', 'x', 'z'], freq=2)
FreqItemset(items=['s', 'y', 'z'], freq=2)
FreqItemset(items=['s', 'x'], freq=3)
FreqItemset(items=['s', 'x', 'z'], freq=2)
FreqItemset(items=['s', 'z'], freq=2)
FreqItemset(items=['t'], freq=3)
FreqItemset(items=['t', 'y'], freq=3)
FreqItemset(items=['t', 'y', 'x'], freq=3)
FreqItemset(items=['t', 'y', 'x', 'z'], freq=3)
FreqItemset(items=['t', 'y', 'z'], freq=3)
FreqItemset(items=['t', 's'], freq=2)
FreqItemset(items=['t', 's', 'y'], freq=2)
FreqItemset(items=['t', '

## Read and parse data in spark

In this project, your function, let's still call it `myParser` for `data.map()` should take a line of string and  return a tuple of

`IP, TIMESTAMP, REQUEST_URL, STATUS`

- `IP` can be a string, 
- `TIMESTAMP` should be a datetime object, refer to [this tutorial](https://stackabuse.com/converting-strings-to-datetime-in-python/) for converting string to datetime object. (note: timezone doesn't matter in this project because all records are in the same timezone, which is server time, so you can ignore it.)
- `REQUEST_URL` is a string
- STATUS is integer, 200, 302 etc.


It's a good habit to test your function `myParser` first, with a smaller dataset and without spark, before proceeding. You can create a 20 line file for this by running this in docker container Linux command line:
```bash
tail -n 20 access.log > small20.log
```

This will create a file "small20.log"

You can view it by running following command in docker container Linux command:
```bash
cat small20.log
```

Now you can test this file (note it's sitting inside `"/home/jovyan/mydir"` ) by following python code in Jupyter notebook:

```python
# Note this test doesn't involve spark
lines = open("/home/jovyan/mydir/small20.log").readlines()
for l in map(myParser, lines):
   print(l)
```




## <font color=red>You code for defining function `myParser` and testing it on small20.log without using spark (you can use code above) here </font>

In [ ]:
# !tail -n 20 access.log > small20.log
# !tail -n 200 access.log > small200.log
# !tail -n 500 access.log > small500.log
# !tail -n 1000 access.log > small1000.log
# !tail -n 2000 access.log > small2000.log
# !tail -n 5000 access.log > small5000.log
# !tail -n 10000 access.log > small10000.log
# !tail -n 50000 access.log > small50000.log
# !tail -n 100000 access.log > small100000.log
# !tail -n 500000 access.log > small500000.log
# !tail -n 1000000 access.log > small1000000.log
# !tail -n 5000000 access.log > small5000000.log
# !tail -n 10000000 access.log > small10000000.log

In [117]:
from datetime import datetime
def myParser2(logdata):
    tmp = logdata.strip().split(' ')

    statlist = []
    
    for j in range(len(tmp)):
        if 'HTTP' in tmp[j]:
            statlist.append(tmp[j+1])
            
    REQUEST_URL = tmp[6]
    
    if len(statlist) == 0:
        STATUS = 0
    else:
        STATUS = statlist[0]
            
    # return str(tmp[0]), tmp[3][1:], str(REQUEST_URL), STATUS
    return str(tmp[0]), datetime.strptime(tmp[3][1:], '%d/%b/%Y:%H:%M:%S'), str(REQUEST_URL), STATUS


In [118]:
lines = open("/home/jovyan/mydir/small20.log").readlines()

for l in map(myParser2, lines):
    print(l)

('91.99.55.165', datetime.datetime(2019, 1, 26, 20, 29, 12), '/m/filter/p2597%2Cb231?page=1', '200')
('151.242.106.253', datetime.datetime(2019, 1, 26, 20, 29, 12), '/image/30181?name=fsr14-55.jpg&wh=max', '200')
('192.15.51.231', datetime.datetime(2019, 1, 26, 20, 29, 13), '/image/267/productModel/150x150', '200')
('13.66.139.0', datetime.datetime(2019, 1, 26, 20, 29, 13), '/product/29746/%D9%85%D8%A7%D8%B4%DB%8C%D9%86-%D8%A7%D8%B5%D9%84%D8%A7%D8%AD-%D8%A8%D8%AF%D9%86-%D9%BE%D8%B1%D9%86%D8%B3%D9%84%DB%8C-%D9%85%D8%AF%D9%84-Princely-Body-Groomer-PR461AT', '200')
('45.79.177.249', datetime.datetime(2019, 1, 26, 20, 29, 13), '/m/browse/evaporative-air-cooler/%DA%A9%D9%88%D9%84%D8%B1-%D8%A2%D8%A8%DB%8C', '200')
('180.94.84.225', datetime.datetime(2019, 1, 26, 20, 29, 13), '/image/get?path=/Image/ofa4bc2n.jpg', '200')
('5.134.161.178', datetime.datetime(2019, 1, 26, 20, 29, 13), '/image/216/brand', '200')
('5.134.161.178', datetime.datetime(2019, 1, 26, 20, 29, 13), '/image/217/brand', '20

After your test, apply your function `myParser` in spark RDD and print out 15 rows （use data.take(15) ）from your large spark data (based upon the large access.log file), note each row will be:

IP, TIMESTAMP, REQUEST_URL, STATUS


## <font color=red>You code to test `myParser` on reading access.log as spark RDD and print out 15 rows here </font>

In [63]:
from pyspark.mllib.fpm import FPGrowth
from pyspark import SparkContext 
sc = SparkContext.getOrCreate()
data = sc.textFile("access.log") 
parsed_data = data.map(myParser2)

In [64]:
parsed_data.take(15)

[('54.36.149.41',
  '22/Jan/2019:03:56:14',
  '/filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,27|%DA%A9%D9%85%D8%AA%D8%B1%20%D8%A7%D8%B2%205%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,p53',
  '200'),
 ('31.56.96.51',
  '22/Jan/2019:03:56:16',
  '/image/60844/productModel/200x200',
  '200'),
 ('31.56.96.51',
  '22/Jan/2019:03:56:16',
  '/image/61474/productModel/200x200',
  '200'),
 ('40.77.167.129',
  '22/Jan/2019:03:56:17',
  '/image/14925/productModel/100x100',
  '200'),
 ('91.99.72.15',
  '22/Jan/2019:03:56:17',
  '/product/31893/62100/%D8%B3%D8%B4%D9%88%D8%A7%D8%B1-%D8%AE%D8%A7%D9%86%DA%AF%DB%8C-%D9%BE%D8%B1%D9%86%D8%B3%D9%84%DB%8C-%D9%85%D8%AF%D9%84-PR257AT',
  '200'),
 ('40.77.167.129',
  '22/Jan/2019:03:56:17',
  '/image/23488/productModel/150x150',
  '200'),
 ('40.77.167.129',
  '22/Jan/2019:03:56:18',
  '/image/45437/productModel/150x150',
  '200'),
 ('40.77.167.129',
  '22/Jan/2019:03:56:18',
  '/image/576/article/100x100',
  '200'),
 ('66.249.66.194

## Filtering the spark data

Now you want to filter your RDD. We completed step 0 below. The remaining steps are 1,2,3.

0. read the access.log into spark (**completed**)
1. filter out all unnecessary lines as highlighted before.
2. Get a list of IPs (let's call it uniqueIPs) who performed the action of adding product(s) to the basket 
3. filter out all records whose IP does not belong to the IP list in step 2.

#### Converting RDD to dataframe

In order to do filtering operations, we want to convert the RDD into spark dataframe. Let's run and study following examples. 

**Note**: in case you get error with `ValueError: Cannot run multiple SparkContexts at once` you need to stop previous `SparkContext` by running
```python
sc.stop()
```

Or you comment the line for `sc = SparkContext('local[*]')`

**Important** Run and **study** the code example below carefully, you might use every line of it, with modification, for the project.

```python
from pyspark.sql import SparkSession
from pyspark import SparkContext
import pyspark.sql.functions as F


sc = SparkContext('local[*]') # comment this in case you already have SparkContext running



dataList = [("James", "Cameron","Sales","NY",90000,34,10000),
    ("Michael", "Johnson","Sales","NY",86000,56,20000),
    ("Robert", "Stromberg","Sales","CA",81000,30,23000),
    ("Maria", "McDonald","Finance","CA",90000,24,23000),
    ("Raman","Pearson","Finance","CA",99000,40,24000),
    ("Scott","Kindel","Finance","NY",83000,36,19000),
    ("Jen","Lewis","Finance","NY",79000,53,15000),
    ("Jeff", "Scott","Marketing","CA",80000,25,18000),
    ("Kumar", "Jobs","Marketing","NY",91000,50,21000)
  ]
rdd=sc.parallelize(dataList) # Note RDD is created from list, similar to creating it from a text file

# *** Now we convert rdd to dataframe
spark = SparkSession(sc) # this is necessary for rdd to be converted to data frame

# converting RDD to DataFrame
df = rdd.toDF(schema = ["first_name", "last_name","department","state","salary","age","bonus"])

# Show schema
df.printSchema()

# Show data frame
df.show(truncate=False)

# Note we create two new columns from grouped data:
# they are renamed as dep_salary, dep_individual_salary
df.groupBy("department").agg(F.sum('salary').alias("dep_salary"), 
          F.collect_list('salary').alias("dep_individual_salary")).show()
```


In [13]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
import pyspark.sql.functions as F

# sc = SparkContext('local[*]') # comment this in case you already have SparkContext running
sc = SparkContext.getOrCreate();

dataList = [("James", "Cameron","Sales","NY",90000,34,10000),
    ("Michael", "Johnson","Sales","NY",86000,56,20000),
    ("Robert", "Stromberg","Sales","CA",81000,30,23000),
    ("Maria", "McDonald","Finance","CA",90000,24,23000),
    ("Raman","Pearson","Finance","CA",99000,40,24000),
    ("Scott","Kindel","Finance","NY",83000,36,19000),
    ("Jen","Lewis","Finance","NY",79000,53,15000),
    ("Jeff", "Scott","Marketing","CA",80000,25,18000),
    ("Kumar", "Jobs","Marketing","NY",91000,50,21000)
  ]
rdd=sc.parallelize(dataList) # Note RDD is created from list, similar to creating it from a text file

# *** Now we convert rdd to dataframe
spark = SparkSession(sc) # this is necessary for rdd to be converted to data frame

# converting RDD to DataFrame
df = rdd.toDF(schema = ["first_name", "last_name","department","state","salary","age","bonus"])

# Show schema
df.printSchema()

# Show data frame
df.show(truncate=False)

# Note we create two new columns from grouped data:
# they are renamed as dep_salary, dep_individual_salary
df.groupBy("department").agg(F.sum('salary').alias("dep_salary"), 
          F.collect_list('salary').alias("dep_individual_salary")).show()

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- state: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- age: long (nullable = true)
 |-- bonus: long (nullable = true)

+----------+---------+----------+-----+------+---+-----+
|first_name|last_name|department|state|salary|age|bonus|
+----------+---------+----------+-----+------+---+-----+
|James     |Cameron  |Sales     |NY   |90000 |34 |10000|
|Michael   |Johnson  |Sales     |NY   |86000 |56 |20000|
|Robert    |Stromberg|Sales     |CA   |81000 |30 |23000|
|Maria     |McDonald |Finance   |CA   |90000 |24 |23000|
|Raman     |Pearson  |Finance   |CA   |99000 |40 |24000|
|Scott     |Kindel   |Finance   |NY   |83000 |36 |19000|
|Jen       |Lewis    |Finance   |NY   |79000 |53 |15000|
|Jeff      |Scott    |Marketing |CA   |80000 |25 |18000|
|Kumar     |Jobs     |Marketing |NY   |91000 |50 |21000|
+----------+---------+----------+-----+-----

## Actual steps for filtering the spark data

Now you write your code to 

1. Convert spark RDD to Data Frame, then filter out all unnecessary lines as highlighted in <font color=orange>orange</font> in the above section `Understand the Web Log Format`.
2. Get a list of IPs (let's call it `uniqueIPs`) who performed the action of adding product(s) to the basket 
3. filter out all records whose IP does not belong to the IP list in step 2.
4. Note step 2 and 3 should not be performed literally. I am using the language for easy understanding. This will be explained further in next section (see `last note`)



## Important resources for wrangling spark data, should you encounter problems

- When testing your code you have to go through many trial and error cycles; It may speed up your coding process by creating a reletively smaller data file (say, 10,000 ~ 30,000 lines) to test your code until it succeeds.
- Please bookmark [this tutorial](https://sparkbyexamples.com/) and study relevant examples (browse the menu on the left) if you have trouble with certain functions.
- You may want to explore `where` or `filter`, `groupBy`, `agg`, `withColumn`, `Date and Timestamp Functions`, `udf` (user defined function)
- **Note** the examples in general ommited import statements. If you have errors saying certain objects not defined, it may be resolved by 
```python
from pyspark.sql.functions import *
from from pyspark.sql.types import *
```
Or you search google with something like `pyspark import udf`
- One useful function to be applied to data frame column is `.isin()` which takes a list and check column elements against that list. see https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.Column.isin.html?highlight=isin
- To convert a spark dataframe column into a python list, you need to use toPandas then pandas function will work, for example to get first name list from above example df:
```python
h = df.select("first_name").toPandas()
huge_list = list( h["first_name"].values)
huge_list
```

- **Last note**: Filtering a spark data frame with a list could be extremely slow if you deal with huge list and big data frame. Standard approach is to use two dataframes and apply [inner join](https://sparkbyexamples.com/pyspark/pyspark-join-explained-with-examples/) for this scenario. So please use inner join or simply use sql syntax. If you stick with `.isin(huge_list)` you get 2 points deduction.

## <font color=red>You code for filtering spark data here </font>

In [7]:
from datetime import datetime
def myParser2(logdata):
    tmp = logdata.strip().split(' ')

    statlist = []
    
    for j in range(len(tmp)):
        if 'HTTP' in tmp[j]:
            statlist.append(tmp[j+1])
            
    REQUEST_URL = tmp[6]
    
    if len(statlist) == 0:
        STATUS = 0
    else:
        STATUS = statlist[0]
            
    # return str(tmp[0]), tmp[3][1:], str(REQUEST_URL), STATUS
    return str(tmp[0]), datetime.strptime(tmp[3][1:], '%d/%b/%Y:%H:%M:%S'), str(REQUEST_URL), STATUS

In [8]:
from pyspark.mllib.fpm import FPGrowth
from pyspark import SparkContext 
sc = SparkContext.getOrCreate()
data = sc.textFile("access.log") 
parsed_data = data.map(myParser2)

In [9]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark import SparkContext
import pyspark.sql.functions as F
from pyspark.sql.functions import col

sc = SparkContext.getOrCreate();

spark = SparkSession(sc)

In [10]:
# converting RDD to DataFrame
df = parsed_data.toDF(schema = ["ip", "timestamp","requesturl","status"])

# Show schema
df.printSchema()

# Show data frame
df.show(5,truncate=True)
# df.show(truncate=False)

root
 |-- ip: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- requesturl: string (nullable = true)
 |-- status: string (nullable = true)

+-------------+-------------------+--------------------+------+
|           ip|          timestamp|          requesturl|status|
+-------------+-------------------+--------------------+------+
| 54.36.149.41|2019-01-22 03:56:14|/filter/27|13%20%...|   200|
|  31.56.96.51|2019-01-22 03:56:16|/image/60844/prod...|   200|
|  31.56.96.51|2019-01-22 03:56:16|/image/61474/prod...|   200|
|40.77.167.129|2019-01-22 03:56:17|/image/14925/prod...|   200|
|  91.99.72.15|2019-01-22 03:56:17|/product/31893/62...|   200|
+-------------+-------------------+--------------------+------+
only showing top 5 rows



In [123]:
filtered_df = df.filter( (col('status') == 200 ) | (col('status').like('3%')) )
filtered_df = filtered_df.filter(~df.requesturl.contains('css') & ~df.requesturl.contains('js'))
filtered_df.show(5)
# #Checking if the like('3%') worked
filtered_df.filter( col('status').like('3%') ).show(5)

+-------------+-------------------+--------------------+------+
|           ip|          timestamp|          requesturl|status|
+-------------+-------------------+--------------------+------+
| 54.36.149.41|2019-01-22 03:56:14|/filter/27|13%20%...|   200|
|  31.56.96.51|2019-01-22 03:56:16|/image/60844/prod...|   200|
|  31.56.96.51|2019-01-22 03:56:16|/image/61474/prod...|   200|
|40.77.167.129|2019-01-22 03:56:17|/image/14925/prod...|   200|
|  91.99.72.15|2019-01-22 03:56:17|/product/31893/62...|   200|
+-------------+-------------------+--------------------+------+
only showing top 5 rows

+-------------+-------------------+--------------------+------+
|           ip|          timestamp|          requesturl|status|
+-------------+-------------------+--------------------+------+
| 54.36.148.87|2019-01-22 03:56:34|/filter/p65%2Cv1%...|   302|
| 5.160.157.20|2019-01-22 03:56:49|     /browse/blu-ray|   301|
| 5.160.157.20|2019-01-22 03:56:49|/filter?f=p12129&...|   301|
|54.36.148.161|

In [124]:
filtered_df.count()

9439664

In [125]:
filtered_df.select('status').distinct().show()

+------+
|status|
+------+
|   200|
|   302|
|   304|
|   301|
+------+



In [12]:
# Get a list of IPs (let's call it uniqueIPs) who performed the action of adding product(s) to the basket
from pyspark.sql.functions import col

# filter the rows with '/basket/add' in 'requesturl' column
ip_addbasket = filtered_df.filter(col('requesturl').contains('/basket/add')).select('ip').distinct()

In [127]:
ip_addbasket.show()
ip_addbasket.count()

+---------------+
|             ip|
+---------------+
|  5.117.242.204|
| 83.122.120.209|
|   89.196.66.66|
|  2.181.243.101|
|178.252.144.130|
|    5.125.13.45|
|  5.114.160.181|
|    2.191.96.86|
|  5.237.115.244|
| 188.166.113.28|
|  5.114.183.125|
|   5.211.134.24|
|  204.18.175.72|
|   185.30.7.254|
| 91.208.165.222|
|178.252.166.116|
|   5.119.221.47|
|  5.121.110.137|
| 204.18.119.253|
|  5.235.227.104|
+---------------+
only showing top 20 rows



2304

In [128]:
# Keep rows only have ips in ip_addbasket
add_basket_ips_df = filtered_df.join(ip_addbasket, on='ip', how='inner')
add_basket_ips_df.show(5,truncate=False)
add_basket_ips_df.count()

+------------+-------------------+-------------------------------------------------------------------------------------+------+
|ip          |timestamp          |requesturl                                                                           |status|
+------------+-------------------+-------------------------------------------------------------------------------------+------+
|81.90.144.56|2019-01-22 07:41:48|/browse/home-appliances/%D9%84%D9%88%D8%A7%D8%B2%D9%85-%D8%AE%D8%A7%D9%86%DA%AF%DB%8C|200   |
|81.90.144.56|2019-01-22 07:41:50|/image/%7B%7BbasketItem.id%7D%7D?type=productModel&wh=50x50                          |200   |
|81.90.144.56|2019-01-22 07:41:51|/image/55/productType/240x180                                                        |200   |
|81.90.144.56|2019-01-22 07:41:51|/image/8/productType/240x180                                                         |200   |
|81.90.144.56|2019-01-22 07:41:51|/image/18342/productType/240x180                                      

1024992

## Prepare Sequential Data for prefixspan 

After you get all data in spark and filter out unnecessary records, we want to prepare the sequential data for prefixspan.

In this project we define a sequence of events as:
 
> A series of URLs requested by one particular user (a user is identified by the IP address), within one day.
e.g., on 26/Jan/2019 a user visited urls at  `/A` then `/B` then at the same timestamp `/C` and `/D`, this will form a sequence as a list of list:

```python
[ ["/A"], ["/B"], ["/C", "/D"] ]
```

So now with your spark data frame (assuming all irrelevant rows filtered out), let's still call it `df`, you want to:


1. Group the dataframe by `(IP, TIMESTAMP)` so actions from same IP and at the same timestamp are grouped together
  - now each group contains one or a set of urls which we define as one event of the sequence
  - you want to apply an aggregation (`agg`) function to create a **list of URLs** for each group, let's call this new aggregated column `EVENTS` and let's call this aggregated data frame `DF2` (refer to reources listed above and refer to the example code above in `Converting RDD to dataframe` section)
2. From `DF2` create a new column called `DATE` which converts the `TIMESTAMP` into a date object so timestamps of the same day have the same values in this column (refer to reources listed above).
3. Since `DATE` column round timestamp to date so if you group the data frame by `(IP, DATE)` you will have in  each group a list of `EVENTS` from each user in each day  -- this is exactly the sequence we defined. So after grouping, just create another aggregated column `SEQUENCE` which consists list of `EVENTS` of the group (refer to the example code above in `Converting RDD to dataframe` section)
4. Finally you want to print out 30 rows from column `SEQUENCE` to verify you did it right.

## <font color=red>You code for `Prepare Sequential Data for prefixspan` here </font>

In [14]:
from pyspark.sql.functions import collect_list

# DF2 = df1mil.groupBy(['IP', 'TIMESTAMP']).agg(collect_list('REQUESTURL').alias('EVENTS'))
DF2 = add_basket_ips_df.groupBy(['ip', 'timestamp']).agg(collect_list('requesturl').alias('EVENTS'))



In [130]:
DF2.show(2, truncate=False)

+--------------+-------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ip            |timestamp          |EVENTS                                                                                                                                                                                                                                      |
+--------------+-------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|10.139.192.130|2019-01-26 13:03:54|[/filter/b43,p63]                                                                                                                             

In [131]:
from pyspark.sql.functions import to_date

DF2 = DF2.withColumn('DATE', to_date('timestamp'))

DF2.show(5)

+--------------+-------------------+--------------------+----------+
|            ip|          timestamp|              EVENTS|      DATE|
+--------------+-------------------+--------------------+----------+
|10.139.192.130|2019-01-26 13:03:54|   [/filter/b43,p63]|2019-01-26|
|10.139.192.130|2019-01-26 13:03:55|[/image/238/brand...|2019-01-26|
|10.139.192.130|2019-01-26 13:03:56|[/image/39/produc...|2019-01-26|
|10.139.192.130|2019-01-26 13:03:57|[/image/3437/prod...|2019-01-26|
|10.139.192.130|2019-01-26 13:03:58|[/image/5947/prod...|2019-01-26|
+--------------+-------------------+--------------------+----------+
only showing top 5 rows



In [16]:
from pyspark.sql.functions import collect_list

# group the dataframe by IP and DATE and aggregate the urls into a list
DF3 = DF2.groupBy(['ip', 'DATE']).agg(collect_list('EVENTS').alias('SEQUENCE'))


In [133]:
DF3.show(3)

+--------------+----------+--------------------+
|            ip|      DATE|            SEQUENCE|
+--------------+----------+--------------------+
|10.139.192.130|2019-01-26|[[/filter/b43,p63...|
| 10.233.251.23|2019-01-24|[[/filter/b256,p4...|
|  10.87.76.147|2019-01-23|[[/browse/air-con...|
+--------------+----------+--------------------+
only showing top 3 rows



In [134]:
DF3.show(3,truncate=False)

+--------------+----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [20]:
from pyspark.sql.functions import col

# DF3.select("SEQUENCE").show(30, truncate=False)
DF3.select("SEQUENCE").show(30)



+--------------------+
|            SEQUENCE|
+--------------------+
|[[/filter/b43,p63...|
|[[/filter/b256,p4...|
|[[/browse/air-con...|
|[[/image/33968?na...|
|[[/image/33980?na...|
|[[/filter/b261], ...|
|[[/], [/settings/...|
|[[/browse/Screen-...|
|[[/], [/, /image/...|
|[[/m/article/618/...|
|[[/basket/add/563...|
|[[/m/browse/washi...|
|[[/product/33596/...|
|[[/product/32446/...|
|[[/m/browse/appli...|
|[[/browse/dishwas...|
|[[/product/16721/...|
|[[/amp-helper-fra...|
|[[/go/CYWO], [/go...|
|[[/image/47987/pr...|
|[[/amp-helper-fra...|
|[[/image/32172?na...|
|[[/filter/b36,p5]...|
|[[/product/34286/...|
|[[/amp-helper-fra...|
|[[/m/product/2933...|
|[[/?utm_campaign=...|
|[[/product/33953/...|
|[[/basket/checkou...|
|[[/settings/logo]...|
+--------------------+
only showing top 30 rows



In [136]:
DF3.count()

3167

## Show time

Now you want to use spark library for prefixspan to mine the sequence data. This is just a simple call:
https://spark.apache.org/docs/latest/api/python//reference/api/pyspark.ml.fpm.PrefixSpan.html

- Note default sequence column in the input data frame is `sequence`, you can change that to match your data frame:
```python
prefixSpan.setSequenceCol('SEQUENCE')
```
- you want to play with `MinSupport` and `MaxPatternLength` in order to get meaningful patterns. Start with higher `MinSupport` value (0.5 is a really high value but you can start with it) and lower `MaxPatternLength` (5 is a good start) to speed up your test run.

- You want to write a summary about your observed patterns in the output.

## <font color=red>You code for doing prefixspan here </font>

In [137]:
from pyspark.ml.fpm import PrefixSpan
from pyspark.sql import Row

prefixSpan = PrefixSpan()
prefixSpan.getMaxLocalProjDBSize()
prefixSpan.setSequenceCol('SEQUENCE')
prefixSpan.getSequenceCol()

prefixSpan.setMinSupport(0.5)

prefixSpan.setMaxPatternLength(5)

prefixSpan.findFrequentSequentialPatterns(DF3).sort("SEQUENCE").show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------+----+
|sequence                                                                                                            |freq|
+--------------------------------------------------------------------------------------------------------------------+----+
|[[/basket/checkout]]                                                                                                |1639|
|[[/basket/view]]                                                                                                    |1617|
|[[/favicon.ico]]                                                                                                    |1603|
|[[/settings/logo]]                                                                                                  |2931|
|[[/settings/logo], [/basket/checkout]]                                                                              |1633|
|[[/sett

In [4]:
from pyspark.ml.fpm import PrefixSpan
from pyspark.sql import Row

In [18]:
prefixSpan = PrefixSpan()
prefixSpan.getMaxLocalProjDBSize()
prefixSpan.setSequenceCol('SEQUENCE')
prefixSpan.getSequenceCol()

prefixSpan.setMinSupport(0.3)

prefixSpan.setMaxPatternLength(30)

prefixSpan.findFrequentSequentialPatterns(DF3).sort("SEQUENCE").show(100,truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+
|sequence                                                                                                                                                                                                                                                    |freq|
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+
|[[/basket/checkout]]                                                                                                                                                                                                       

In [19]:
prefixSpan = PrefixSpan()
prefixSpan.getMaxLocalProjDBSize()
prefixSpan.setSequenceCol('SEQUENCE')
prefixSpan.getSequenceCol()

prefixSpan.setMinSupport(0.2)

prefixSpan.setMaxPatternLength(50)

prefixSpan.findFrequentSequentialPatterns(DF3).sort("SEQUENCE").show(250,truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+
|sequence                                                                                                                                                                                                                                               |freq|
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+
|[[/]]                                                                                                                                                                                                                                     

## <font color=red>You summary and clusions here </font>

In this data mining report, I analyzed a substantial dataset containing approximately 10 million rows. Due to the project's limitations, we worked on a single machine, which prevented leveraging PySpark's distributed processing capabilities. As a result, the processing time was considerably longer than anticipated.

Findings: <br>
I experimented with various MinSupport and MaxPatternLength parameters and identified some interesting patterns in the web log data. For instance, customers often review their carts before proceeding to checkout, indicating a preference for reviewing items rather than using the "buy now" option.

Additionally, many logs show that URLs containing image files (.png) preceded adding an item to the basket. This suggests that customers are influenced by product images when deciding to make a purchase. Therefore, optimizing product images could have a significant impact on sales, as most "adding to basket" actions occurred after viewing these images.

I also observed that some checkouts followed browsing on Instagram or Telegram, indicating that social media marketing could be a promising avenue to explore based on this dataset.

However, it is worth noting that web logs may not be the most suitable data source for identifying purchase patterns or sequential patterns. The dataset contained a substantial amount of noise, including irrelevant information such as JavaScript and CSS logs. To improve future data mining efforts, I recommend designing the website to log specific user actions related to adding items to the basket. This would enable more accurate analysis and, in turn, more effective decision-making based on the mined data.

# DELIVER (individually)



Submit on canvas:

* This notebook with all results shown
* If you discussed with classmates and get inspirations during the project write down his/her names in next section. No points deductions but you have to be honest.



<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>
<font size="+2" color="Black">Kenny Zhang